# Medical POS
Identify the place of service for all medical claims extracted from raven header.

**Script**
* [scripts/cld/medical_pos.ipynb](./scripts/cld/medical_pos.ipynb)

**Prior Script(s)**
* [scripts/de/raven_header.ipynb](./scripts/de/raven_header.ipynb)

**Parameters**
* `in/pld/medical_pos.xlsx[param]`

**Input**
* `coh_pt`
* `pld_pos_ref`: `in/pld/medical_pos.xlsx[pos]`
* `de_raven_header`

**Output**  
* `pld_med_pos`

**Review**
* [scripts/cld/medical_pos.html](./scripts/cld/medical_pos.html)

In [1]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Reference Table
Upload reference table used to group place of service

**Input**  
  * `in/cld/medical_pos.xlsx[ref]`

**Output**  
* `cld_med_pos_ref`


In [2]:
#Upload reference table from excel to snowflake and review snowflake output
df = pd.read_excel('../../in/cld/medical_pos.xlsx', sheet_name='pos', skiprows=4, dtype=str)

#Strip white space and make referrable columns uppercase
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df[['claim_type_code','pos_marker']] =  \
    df[['claim_type_code','pos_marker']].apply(lambda x: x.str.upper() if x.dtype == "object" else x)

#Upload to snowflake
snow.drop_table("cld_med_pos_ref")
snow.upload_dataframe(df,"cld_med_pos_ref")
del df
snow.select("SELECT * FROM cld_med_pos_ref")

DROP TABLE IF EXISTS ref_db.semi_custom.cld_med_pos_ref;
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Table ref_db.semi_custom.cld_med_pos_ref dropped! (╯°□°）╯︵ ┻━┻
Upload into ref_db.semi_custom.cld_med_pos_ref successful! ┬──┬◡ﾉ(°-°ﾉ)


,claim_type_code,pos_marker,pos,pos_group
0,P,01,Pharmacy,Pharmacy
1,P,03,School,Other
2,P,04,Homeless Shelter,Other
3,P,05,Indian Health Center – Freestanding,Other
4,P,06,Indian Health Center – Provider Based,Other
5,P,09,Correctional Facility,Other
6,P,11,Office,Office
7,P,12,Home,Home Health
8,P,13,Assisted Living,Long Term Care
9,P,14,Group Home,Group Home


# Extract Data
Extract subset of raven diagnosis for the patients of interest between specified date ranges

**Parameters**
  * `min_dx_dt`
  * `max_dx_dt`
  
**Input**
  * `coh_cohort`
  * `de_raven_header`
  * `cld_comorbid_ref`
  
**Output**  
* `cld_med_pos`

| Column | Description |
| --- | --- |
| claim_id | |
| patient_id | |
| year_of_service | |
| claim_type_code | |
| type_bill | |
| pos | |
| pos_group | |

In [3]:
%%read_sql
--Create raven diagnosis table
DROP TABLE IF EXISTS cld_med_pos;
CREATE TRANSIENT TABLE cld_med_pos AS 
  SELECT head.claim_id, 
         head.patient_id, 
         head.year_of_service,
         head.claim_type_code,
         head.type_bill,
         ref.pos,
         ref.pos_group
    FROM de_raven_header head
         LEFT JOIN cld_med_pos_ref ref
                ON head.claim_type_code = ref.claim_type_code
                   AND Left(head.type_bill,2) = Left(ref.pos_marker,2); 

Query started at 04:36:42 PM Eastern Daylight Time; Query executed in 0.03 mQuery started at 04:36:44 PM Eastern Daylight Time; Query executed in 0.09 m

,status
0,Table CLD_MED_POS successfully created.


In [4]:
%%read_sql
--Review pos_group
SELECT pos_group,
       Count(*) AS CNT
  FROM cld_med_pos
 group by pos_group
 ORDER BY CNT desc

Query started at 04:36:49 PM Eastern Daylight Time; Query executed in 0.06 m

,pos_group,cnt
0,Office,1362518
1,Outpatient Hospital,1158103
2,None,750023
3,Home Health,388057
4,Inpatient Hospital,318968
5,Other,286715
6,Emergency Room,139885
7,Ambulatory Surgery Center,84101
8,Long Term Care,70731
9,Transport,20288


In [5]:
%%read_sql
--Review combination of POS and pos_group
SELECT pos,
       pos_group,
       Count(*) AS CNT
  FROM cld_med_pos
 group by pos, pos_group
 ORDER BY CNT desc

Query started at 04:36:53 PM Eastern Daylight Time; Query executed in 0.06 m

,pos,pos_group,cnt
0,Office,Office,1357968
1,Outpatient Hospital,Outpatient Hospital,1110567
2,None,None,750023
3,Inpatient Hospital,Inpatient Hospital,318968
4,Home,Home Health,317543
5,Independent Laboratory,Other,167237
6,Emergency Room,Emergency Room,139885
7,Ambulatory Surgery Center,Ambulatory Surgery Center,84101
8,Home Health,Home Health,70514
9,Off Campus-Outpatient Hospital,Outpatient Hospital,47536
